Simulating Bubble Shooter Game Play

This code is about simulating a basic bubble shooter game play
and with this way you can adjust your difficulty.
The board generation is randomly distributed but in general 
there is already a level.txt so pre-defined board which has 
tactics for the difficulty curve but we keep things basic in here.



In [1]:
import numpy as np
import math

class simulation():
    @staticmethod
    def define_ball_colors(colors):
        #Define ball colors array from number of colors
        return list(range(1, colors+1, 1))
    
    @staticmethod
    def generate_board(rows, cols, colors):
        # Define the size of the matrix
        #rows = board height
        #cols = board width
        
        # Generate the matrix with randomly distributed 1, 2, and 3
        board = np.random.choice(board_colors, size=(rows, cols))
        #We flipped board here because in general a level.txt file starts from the top
        #Using this know-how we assumed our generated board started from top.
        board = np.flipud(board)
        
        return board
        
    @staticmethod
    def euclidean_distance(point1, point2):
        #calculating euclidean distance
        return math.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2)
    
    @staticmethod
    def choose_ball(board = np.zeros([5,5]), ball_colors = 3 ):
        #This function choose the ball that is in board.
        #If the ball is not in board then it shouldn't be chosen.
        color_lst = simulation.define_ball_colors(ball_colors)
        color_counts = simulation.count_colors(board)
        exclude_list = []
        for color in color_lst:
            if color not in color_counts.keys():
                exclude_list.append(int(color))

            for color in color_lst:
                for color in board:         
                    if (color in board):
                        if len(exclude_list) > 0:  
                            color_lst_np = np.array(color_lst)
                            new_color_lst = color_lst_np[~np.isin(color_lst_np, exclude_list)].tolist()
                        else:
                            new_color_lst = color_lst
                        if len(new_color_lst) > 0:
                            catapult = np.random.choice(new_color_lst) 
                        else: 
                            break
            return catapult
        
            
    @staticmethod
    def find_connected(row, col, target, rows, cols, visited):
        #This function searches the connections of the ball that matched on the board.
        #Also the search method is diagonally.
        
        # Stop the recursion if out of bounds or not matching the target
        if not (0 <= row < rows and 0 <= col < cols):
            return set()
        if (row, col) in visited or board[row, col] != target:
            return set()
        for vr, vc in visited:
            if simulation.euclidean_distance((vr, vc),(row,col)) > 2:
                return set()
            
                
        # Mark the current cell as visited
        visited.add((row, col))

        # Initialize the set of connected cells with the current cell
        connected = {(row, col)}

        # Directions: right, down, left, up, and down-right (diagonal)
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]
        for dr, dc in directions:
            connected |= simulation.find_connected(row + dr, col + dc, target, rows, cols, visited)
            break
        return connected

    
    @staticmethod
    def process_shoot(board, target):
        #This function process the shoot using by find_connected function
        
        rows, cols = board.shape
        visited = set()
        for i in range(rows):
            if board[i].sum() > 0:  # Check if the row contains non-zero elements
                for j in range(cols):
                    if board[i, j] == target:
                        connected_cells = simulation.find_connected(i, j, target, rows, cols, visited)
                        if connected_cells:
                            # Set all connected cells to zero
                            for cell in connected_cells:
                                board[cell[0], cell[1]] = 0

        return board
        
            

    @staticmethod
    def update_board(board, target):
        new_board = simulation.process_shoot(board, target)


        return new_board

    
    @staticmethod
    def color_distribution(board, board_colors):
        #Generate color density
        counter = 0
        color_distribution = {}
        for color in board_colors:
            for array in board:
                if color in array:
                    counter += 1
            color_distribution[color] = counter
            
        return color_distribution  
    
    @staticmethod
    def color_density(color_distribution):
        #Generate color density
        density = {}
        total_sum = sum(color_distribution.values())
        for color in color_distribution.keys():
            density[color] = color_distribution[color] / total_sum
        return density
    
    @staticmethod
    def count_colors(board):
        #Count colors in board
        count_dict = {}
        if isinstance(board, np.ndarray) and board.ndim == 2:
            for row in board:
                for item in row:
                    item = int(item)
                    count_dict[item] = count_dict.get(item, 0) + 1
        else:
            pass
    
        return count_dict
    
    @staticmethod
    def play_game(board, ball_colors, total_moves, rows, cols, shoot):
        board_colors = simulation.define_ball_colors(ball_colors)
        result = int()
        
        pick_ball = simulation.choose_ball(board, ball_colors)
        board = simulation.update_board(board, pick_ball)
        #print(pick_ball)
        #print(board)
        if 0 in simulation.count_colors(board).keys(): 
            if simulation.count_colors(board)[0] == rows * cols:
                result = 1
                
            
        if shoot == total_moves:
            if board.sum() > 0:
                result = 0
                
        return result
            
                  
            
                    
            
        

In [2]:
def playing(trial_count, board, ball_colors, total_moves, rows, cols):
    result = []
    for trial in range(trial_count):
        # It's crucial that the board used in play_game is not altered,
        # or if it is, it should be reset here.
        board_copy = np.copy(board)  # Make a deep copy of the board for each trial

        for shoot in range(1, total_moves + 1):
            # Simulate one instance of the game.
            game_result = simulation.play_game(board_copy, ball_colors, total_moves, rows, cols, shoot)
        
        result.append(game_result)
    
    # Calculate and return the probability of success
    probability_of_success = sum(result) / trial_count
    print("Probability of Success: ", probability_of_success)
    return probability_of_success

In [4]:
ball_colors = 4
total_moves = 53
rows = 15
cols = 5
trial_count = 1000
board_colors = simulation.define_ball_colors(ball_colors)
board = simulation.generate_board(rows, cols, ball_colors)
print("Game Play Board:\n", board)
playing(trial_count, board, ball_colors, total_moves, rows, cols)

Game Play Board:
 [[4 4 4 4 1]
 [3 3 4 1 1]
 [3 4 2 2 1]
 [4 3 2 4 1]
 [1 4 2 3 2]
 [4 3 1 4 2]
 [3 4 2 2 3]
 [2 4 1 2 2]
 [3 4 1 3 3]
 [1 2 2 2 2]
 [2 1 1 4 4]
 [3 4 3 1 3]
 [3 4 3 1 1]
 [2 1 2 3 2]
 [3 4 3 1 2]]
Probability of Success:  0.798


0.798